<a href="https://colab.research.google.com/github/JoaoVitorDeOliveira/blueavenir/blob/master/BlueAvenir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_customer = pd.read_csv('Customer Data.csv', header=0, sep=';')


In [3]:
df_action = pd.read_csv('Action Space.csv', header=0, sep=';')


In [4]:
df_restriction = pd.read_csv('Tec Restrictions.csv', header=0, sep=';')
from google.colab import data_table
data_table.DataTable(df_restriction, include_index=False)


,KOMBI_ID,TECHNOLOGIE_GRUPPE,BANDBREITE_MB,ENDGERAET
0,1,BSA,50,AVM FRITZ!Box 7430
1,2,BSA,100,AVM FRITZ!Box 7430
2,3,BSA,50,AVM FRITZ!Box 7490
3,4,BSA,100,AVM FRITZ!Box 7490
4,5,BSA,50,AVM FRITZ!Box 7530
...,...,...,...,...
93,94,G.fast,50,AVM FRITZ!Box 7583
94,95,G.fast,100,AVM FRITZ!Box 7583
95,96,G.fast,250,AVM FRITZ!Box 7583
96,97,G.fast,500,AVM FRITZ!Box 7583


In [6]:
df_actions_column = df_customer.drop(columns=['CUSTOMER_ID','PRODUCT_RANGE','PRODUCT','CONTRACT_START_DT','SPEED_MB','TECH','BEST_TECH','ROUTER','ROUTER_TYPE'])

for action_column in df_actions_column.columns:

  # Conditions for PROMOTION_TELEFONFLAT-----------------
  is_single_flat = (df_customer['PRODUCT'] == 'Single Flat')
  is_double_flat = (df_customer['PRODUCT'] == 'Double Flat')
  has_telefonflat = int(df_action['PROMOTION_TELEFONFLAT'][df_action['ACTION_ID'] == action_column])

  # OBS: if Single Flats are also eligible for Actions = 1 remove has_telefonflat
  PROMOTION_TELEFONFLAT_FIRST_CONDITION = (is_single_flat & 
                                           (has_telefonflat == False)) 
  
  PROMOTION_TELEFONFLAT_SECOND_CONDITION = ((is_double_flat) & 
                                            has_telefonflat)

# Conditions for SPEED_MB_TARGET ----------------------------
  
  df_customer[action_column] = np.where((PROMOTION_TELEFONFLAT_FIRST_CONDITION | PROMOTION_TELEFONFLAT_SECOND_CONDITION), 1, 0)

df_customer

,CUSTOMER_ID,PRODUCT_RANGE,PRODUCT,CONTRACT_START_DT,SPEED_MB,TECH,BEST_TECH,ROUTER,ROUTER_TYPE,A_01,A_02,A_03,A_04,A_05,A_06,A_07,A_08,A_09,A_10,A_11,A_12,A_13,A_14,A_15,A_16,A_17,A_18,A_19,A_20,A_21,A_22,A_23,A_24,A_25,A_26,A_27,A_28,A_29,A_30,A_31,A_32,A_33,A_34,A_35,A_36,A_37,A_38,A_39,A_40,A_41,A_42,A_43,A_44,A_45,A_46,A_47,A_48,A_49,A_50,A_51,A_52,A_53,A_54,A_55,A_56,A_57,A_58,A_59,A_60
0,329 08 509,2015,Single Flat,10.11.2019,100,Cable,Cable,Arris TG2492S,basic,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0
1,235 09 840,2018,Double Flat,08.03.2018,50,FTTC,FTTH,AVM FRITZ!Box 7430,basic,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1
2,192 85 309,2015,Tripple Flat,21.07.2019,25,BSA,Cable,n.a.,n.a.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,420 95 309,2012,Single Flat,22.09.2019,50,FTTH,FTTH,AVM FRITZ!Box 7390,basic,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0
4,309 50 932,2018,Double Flat,15.01.2020,18,FTTC,FTTH,AVM FRITZ!Box 7590,premium,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1
5,239 85 092,2018,Single Flat,18.12.2018,25,BSA,FTTH,AVM FRITZ!Box 7583,premium,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0
6,230 95 081,2012,Double Flat,05.05.2019,100,FTTH,FTTH,AVM FRITZ!Box 7390,basic,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1
7,595 80 238,2015,Double Flat,21.11.2019,100,FTTH,FTTH,n.a.,n.a.,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1
8,239 50 098,2018,Double Flat,07.06.2019,50,FTTH,FTTH,n.a.,n.a.,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1
9,915 80 908,2018,Single Flat,23.08.2019,25,BSA,G.fast,AVM FRITZ!Box 7530,basic,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0


In [10]:
df_actions_column = df_customer.drop(columns=['CUSTOMER_ID','PRODUCT_RANGE','PRODUCT','CONTRACT_START_DT','SPEED_MB','TECH','BEST_TECH','ROUTER','ROUTER_TYPE'])

for action_column in df_actions_column.columns:
  is_actions_with_technolgy_switch_enable = int(df_action['TECHNOLOGY_SWITCH'][df_action['ACTION_ID'] == action_column])
  for row in df_customer.itertuples():

    # Conditions for PROMOTION_TELEFONFLAT-----------------
    is_single_flat = (row.PRODUCT == 'Single Flat')
    is_double_flat = (row.PRODUCT  == 'Double Flat')
    has_telefonflat = int(df_action['PROMOTION_TELEFONFLAT'][df_action['ACTION_ID'] == action_column])

    # OBS: if Single Flats are also eligible for Actions = 1 remove has_telefonflat
    PROMOTION_TELEFONFLAT_FIRST_CONDITION = (is_single_flat)    
    PROMOTION_TELEFONFLAT_SECOND_CONDITION = ((is_double_flat) & has_telefonflat)
    
    df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID, action_column] = int(PROMOTION_TELEFONFLAT_FIRST_CONDITION | PROMOTION_TELEFONFLAT_SECOND_CONDITION)

    # Conditions for SPEED_MB_TARGET ----------------------------
    is_current_speed_higher = row.SPEED_MB < int(df_action['SPEED_MB_TARGET'][df_action['ACTION_ID'] == action_column])    
    is_current_tech_lower_then_maximum = row.SPEED_MB < max(df_restriction['BANDBREITE_MB'][df_restriction['TECHNOLOGIE_GRUPPE'] == row.TECH])
    is_best_tech_lower_then_maximum = row.SPEED_MB < max(df_restriction['BANDBREITE_MB'][df_restriction['TECHNOLOGIE_GRUPPE'] == row.BEST_TECH])
    SPEED_MB_TARGET_FIRST_CONDITION = (is_current_speed_higher & is_current_tech_lower_then_maximum & is_best_tech_lower_then_maximum)

    if(df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID][action_column].values[0]):
      df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID, action_column] = int(SPEED_MB_TARGET_FIRST_CONDITION)
    
    # Conditions for TECHNOLOGY_SWITCH ----------------------------
    not_elegible_for_technology_switch = int(row.TECH == row.BEST_TECH)

    if(df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID][action_column].values[0]):
      if(not_elegible_for_technology_switch & is_actions_with_technolgy_switch_enable == 0):
        df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID, action_column] = 1
      else:
        df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID, action_column] = 0

    # Conditions for ROUTER_SWITCH ----------------------------
    if(df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID][action_column].values[0]):
      if(row.ROUTER == 'n.a.'):
        df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID, action_column] = 1
      else:
        customer_router_max_speed = max(df_restriction['BANDBREITE_MB'][df_restriction['ENDGERAET'] == row.ROUTER])
        routers_status = int(df_action['ROUTER_SWITCH'][df_action['ACTION_ID'] == action_column])
        action_router_type = df_action['ROUTER_TYPE'][df_action['ACTION_ID'] == action_column].values[0]
        action_router_speed = int(df_action['SPEED_MB_TARGET'][df_action['ACTION_ID'] == action_column])

        if((customer_router_max_speed >= action_router_speed) & (row.ROUTER_TYPE == 'basic')):
          df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID, action_column] = routers_status
        elif((customer_router_max_speed >= action_router_speed) & (action_router_type == 'premium')):
          df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID, action_column] = 1
        else:
          df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID, action_column] = 0

df_customer

,CUSTOMER_ID,PRODUCT_RANGE,PRODUCT,CONTRACT_START_DT,SPEED_MB,TECH,BEST_TECH,ROUTER,ROUTER_TYPE,A_01,A_02,A_03,A_04,A_05,A_06,A_07,A_08,A_09,A_10,A_11,A_12,A_13,A_14,A_15,A_16,A_17,A_18,A_19,A_20,A_21,A_22,A_23,A_24,A_25,A_26,A_27,A_28,A_29,A_30,A_31,A_32,A_33,A_34,A_35,A_36,A_37,A_38,A_39,A_40,A_41,A_42,A_43,A_44,A_45,A_46,A_47,A_48,A_49,A_50,A_51,A_52,A_53,A_54,A_55,A_56,A_57,A_58,A_59,A_60
0,329 08 509,2015,Single Flat,10.11.2019,100,Cable,Cable,Arris TG2492S,basic,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,235 09 840,2018,Double Flat,08.03.2018,50,FTTC,FTTH,AVM FRITZ!Box 7430,basic,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,192 85 309,2015,Tripple Flat,21.07.2019,25,BSA,Cable,n.a.,n.a.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,420 95 309,2012,Single Flat,22.09.2019,50,FTTH,FTTH,AVM FRITZ!Box 7390,basic,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,309 50 932,2018,Double Flat,15.01.2020,18,FTTC,FTTH,AVM FRITZ!Box 7590,premium,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1
5,239 85 092,2018,Single Flat,18.12.2018,25,BSA,FTTH,AVM FRITZ!Box 7583,premium,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1
6,230 95 081,2012,Double Flat,05.05.2019,100,FTTH,FTTH,AVM FRITZ!Box 7390,basic,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,595 80 238,2015,Double Flat,21.11.2019,100,FTTH,FTTH,n.a.,n.a.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,239 50 098,2018,Double Flat,07.06.2019,50,FTTH,FTTH,n.a.,n.a.,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,915 80 908,2018,Single Flat,23.08.2019,25,BSA,G.fast,AVM FRITZ!Box 7530,basic,0,1,1,0,1,1,0,1,1,0,1,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,1,1,0,1,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
from google.colab import files
df_customer.to_csv('blue_avenir_second_observation.csv') 
files.download('blue_avenir_second_observation.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Conditions for SPEED_MB_TARGET ----------------------------

for action_column in df_actions_column.columns:
  for row in df_customer.itertuples():
    is_current_speed_higher = row.SPEED_MB < int(df_action['SPEED_MB_TARGET'][df_action['ACTION_ID'] == action_column])    
    is_current_tech_lower_then_maximum = row.SPEED_MB < max(df_restriction['BANDBREITE_MB'][df_restriction['TECHNOLOGIE_GRUPPE'] == row.TECH])
    is_best_tech_lower_then_maximum = row.SPEED_MB < max(df_restriction['BANDBREITE_MB'][df_restriction['TECHNOLOGIE_GRUPPE'] == row.BEST_TECH])
    SPEED_MB_TARGET_FIRST_CONDITION = (is_current_speed_higher & is_current_tech_lower_then_maximum & is_best_tech_lower_then_maximum)
    df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID, action_column] = int(SPEED_MB_TARGET_FIRST_CONDITION)

df_customer

In [ ]:
# Conditions for TECHNOLOGY_SWITCH ----------------------------

for action_column in df_actions_column.columns:
  is_actions_with_technolgy_switch_enable = int(df_action['TECHNOLOGY_SWITCH'][df_action['ACTION_ID'] == action_column])
  for row in df_customer.itertuples():
    not_elegible_for_technology_switch = int(row.TECH == row.BEST_TECH)

    if(not_elegible_for_technology_switch & is_actions_with_technolgy_switch_enable == 0):
      df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID, action_column] = 1
    else:
      df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID, action_column] = 0

df_customer 

    

In [235]:
# Conditions for ROUTER_SWITCH ----------------------------

for action_column in df_actions_column.columns:
  for row in df_customer.itertuples():
    if(row.ROUTER == 'n.a.'):
      df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID, action_column] = 1
    else:
      customer_router_max_speed = max(df_restriction['BANDBREITE_MB'][df_restriction['ENDGERAET'] == row.ROUTER])
      routers_status = int(df_action['ROUTER_SWITCH'][df_action['ACTION_ID'] == action_column])
      action_router_type = df_action['ROUTER_TYPE'][df_action['ACTION_ID'] == action_column].values[0]
      action_router_speed = int(df_action['SPEED_MB_TARGET'][df_action['ACTION_ID'] == action_column])

      if((customer_router_max_speed >= action_router_speed) & (row.ROUTER_TYPE == 'basic')):
        df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID, action_column] = routers_status
      elif((customer_router_max_speed >= action_router_speed) & (action_router_type == 'premium')):
        df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID, action_column] = 1
      else:
        df_customer.loc[df_customer.CUSTOMER_ID == row.CUSTOMER_ID, action_column] = 0

df_customer.loc[df_customer.CUSTOMER_ID == '329 08 509']['A_01'].values[0]

0